In [26]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras 
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import glob
import os

In [32]:
from sklearn.datasets import load_files
import numpy as np

train_dir = r'C:\Users\deVam\fruits-360\Training'
test_dir = r'C:\Users\deVam\fruits-360\Test'

def load_dataset(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np.array(data['target'])
    target_labels = np.array(data['target_names'])
    return files,targets,target_labels
    
x_train, y_train,target_labels = load_dataset(train_dir)
x_test, y_test,_ = load_dataset(test_dir)
print('Loading complete!')

print('Training set size : ' , x_train.shape[0])
print('Testing set size : ', x_test.shape[0])

Loading complete!
Training set size :  67692
Testing set size :  22688


In [33]:
no_of_classes = len(np.unique(y_train))
no_of_classes

131

In [34]:
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train,no_of_classes)
y_test = np_utils.to_categorical(y_test,no_of_classes)

In [36]:
from sklearn.model_selection import  train_test_split
x_test,x_valid,y_test,y_valid=train_test_split(x_test,y_test,test_size=0.2,random_state=42)

In [38]:
x_valid.shape, y_valid.shape

((4538,), (4538, 131))

In [39]:
x_test.shape,y_test.shape

((18150,), (18150, 131))

In [40]:
from keras.preprocessing.image import array_to_img, img_to_array, load_img

def convert_image_to_array(files):
    images_as_array=[]
    for file in files:
        # Convert to Numpy Array
        images_as_array.append(img_to_array(load_img(file)))
    return images_as_array

x_train = np.array(convert_image_to_array(x_train))
print('Training set shape : ',x_train.shape)

x_valid = np.array(convert_image_to_array(x_valid))
print('Validation set shape : ',x_valid.shape)

x_test = np.array(convert_image_to_array(x_test))
print('Test set shape : ',x_test.shape)

print('1st training image shape ',x_train[0].shape)

Training set shape :  (67692, 100, 100, 3)
Validation set shape :  (4538, 100, 100, 3)
Test set shape :  (18150, 100, 100, 3)
1st training image shape  (100, 100, 3)


In [41]:
x_train = x_train.astype('float32')/255
x_valid = x_valid.astype('float32')/255
x_test = x_test.astype('float32')/255

In [48]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,AveragePooling2D
from keras.layers import Activation, Dense, Flatten, Dropout,Activation,BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras import backend as K

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size = (3,3),input_shape=(100,100,3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(AveragePooling2D(pool_size=(2,2)))

model.add(Conv2D(filters = 32,kernel_size = (3,3),activation = 'relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 32,kernel_size = (3,3),activation = 'relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())

model.add(Dense(512,activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(256,activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(131,activation = 'softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 100, 100, 16)      448       
_________________________________________________________________
activation_2 (Activation)    (None, 100, 100, 16)      0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 100, 100, 16)      64        
_________________________________________________________________
average_pooling2d_2 (Average (None, 50, 50, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 50, 50, 32)        4640      
_________________________________________________________________
batch_normalization_5 (Batch (None, 50, 50, 32)        128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 25, 25, 32)       

In [49]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [50]:


learned_weights = ModelCheckpoint(filepath = 'fruits_360_weights.hdf5', verbose = 1, save_best_only = True)

history = model.fit(x_train,y_train,
        batch_size = 32,
        epochs=30,
        validation_data=(x_valid, y_valid),
        callbacks = [learned_weights],
        verbose=2, shuffle=True)

Train on 67692 samples, validate on 4538 samples
Epoch 1/30
 - 550s - loss: 1.5209 - accuracy: 0.6220 - val_loss: 0.5963 - val_accuracy: 0.8795

Epoch 00001: val_loss improved from inf to 0.59632, saving model to fruits_360_weights.hdf5
Epoch 2/30
 - 495s - loss: 0.2866 - accuracy: 0.9185 - val_loss: 0.5061 - val_accuracy: 0.9152

Epoch 00002: val_loss improved from 0.59632 to 0.50606, saving model to fruits_360_weights.hdf5
Epoch 3/30
 - 491s - loss: 0.1581 - accuracy: 0.9554 - val_loss: 0.3822 - val_accuracy: 0.9348

Epoch 00003: val_loss improved from 0.50606 to 0.38215, saving model to fruits_360_weights.hdf5
Epoch 4/30
 - 492s - loss: 0.1149 - accuracy: 0.9684 - val_loss: 0.2448 - val_accuracy: 0.9533

Epoch 00004: val_loss improved from 0.38215 to 0.24485, saving model to fruits_360_weights.hdf5
Epoch 5/30
 - 496s - loss: 0.0885 - accuracy: 0.9768 - val_loss: 0.2196 - val_accuracy: 0.9504

Epoch 00005: val_loss improved from 0.24485 to 0.21964, saving model to fruits_360_weights.

In [51]:
model.load_weights('fruits_360_weights.hdf5')

In [52]:
accuracy = model.evaluate(x_test, y_test, verbose=0)

In [53]:
print(accuracy)

[0.16175491615858167, 0.9817630648612976]
